In [ ]:
from pathlib import Path
from datetime import date
from getpass import getpass
from IPython.display import Image
import instructor
from pydantic import BaseModel
from openai import OpenAI
import pandas as pd
from dotenv import load_dotenv
import os
from pycaret.regression import  load_model, predict_model


In [68]:
# załadowanie .env i wczytanie klucza
load_dotenv()
key = os.getenv("OPENAI_API_KEY")

# schemat odpowiedzi AI
class Desc_of_men(BaseModel):
    
    Age_category: int
    Gender: int
    Time_5_km: int
    Info: str

# dane wejściowe 
input="jestem mężczyzną, mam 100 lat przebiegam 5km w 30 minut" 

# funkcja  - pytanie do AI
def ask_AI (input):
    instructor_openai_client = instructor.from_openai(OpenAI(api_key=key))

    resp_AI = instructor_openai_client.chat.completions.create(
        model="gpt-4o-mini",
        response_model=Desc_of_men,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": """Wypełnij klasę Desc_of_men  
                        wiek zaokrąglony do pełnych dziesięcioleci w dół i czas przebiegnięcia 5 km przeliczony na sekundy,
                        jeśli udało się w 100% wypełnić klasę w info wpisz ok jeśli się nie udało to w info wpisz jakich danych 
                        nie udało Ci się uzyskać, nie szacuj danych jednego rodzaju na podstawie innego rodzaju
                        Twój input: """+ input,
                    },
                    # {
                    #     "type": "image_url",
                    #     "image_url": {
                    #         "url": image_data,
                    #         "detail": "high"
                    #     },
                    # },
                ],
            },
        ],
    )
    return resp_AI

# funkcja sprawdzenie odpowiedzi z AI i dopytka lub podanie wyniku przetworzenia modelu

df = ask_AI(input)

data_list = [df]
# print(data_list)
df = pd.DataFrame([data.dict() for data in data_list])



In [69]:
df

,Age_category,Gender,Time_5_km,Info
0,100,1,1800,ok


In [90]:
def check_and_predict(df):
    # Sprawdzenie wartości w wierszu 0 kolumny 'Info'
    if df.loc[0, 'Info'] == "ok":
        # Wartość 'Info' jest 'ok', więc możemy przejść do predykcji
        # 1. Załaduj zapisany model
        model = load_model('best_time_model')  # Upewnij się, że model jest w katalogu roboczym i bez rozszerzenia '.pkl'

        # 2. Przygotuj nowe dane do predykcji
        new_data = df

        # 3. Wykonaj predykcje
        predictions = predict_model(model, data=new_data)

        # Zmień nazwę kolumny predykcji na 'Predicted_time_of_halfmarathon'
        predictions = predictions.rename(columns={'prediction_label': 'Predicted_time_of_halfmarathon'})

        # Wyświetl wynik predykcji
        return predictions
    else:
        # Wartość 'Info' nie jest 'ok', więc informujemy użytkownika o brakujących danych
        missing_info = df.loc[0, 'Info']
        print(f"Opowiedz więcej o: {missing_info}")

In [ ]:
a=check_and_predict(df)



Transformation Pipeline and Model Successfully Loaded


In [92]:
print(a)

   Age_category  Gender  Time_5_km Info  Predicted_time_of_halfmarathon
0           100       1       1800   ok                     7967.910952


In [100]:
def convert_seconds(total_seconds):
    hours = int(total_seconds // 3600)          # Pełne godziny
    minutes = int((total_seconds % 3600) // 60) # Pełne minuty
    seconds = int(total_seconds % 60)           # Pełne sekundy
    return hours, minutes, seconds


# Przykład użycia
total_seconds = a.loc[0, 'Predicted_time_of_halfmarathon']
hours, minutes, seconds = convert_seconds(total_seconds)
print(f"{hours} godzin, {minutes} minut i {seconds} sekund Powodzenia w biegu!")

2 godzin, 12 minut i 47 sekund Powodzenia w biegu!
